In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
reldir = 'C:\\Users\\shefa\\OneDrive\\CS6220\\msd.recsys\\datasets\\'

In [3]:
standardized_df = pd.read_pickle(reldir + 'combined.csv', compression='zip')

Resource: https://colab.research.google.com/github/nipunbatra/blog/blob/master/_notebooks/2017-12-29-neural-collaborative-filtering.ipynb#scrollTo=sjB5374iImJd;
https://www.kaggle.com/jamesloy/deep-learning-based-recommender-systems

In [4]:
master_df = standardized_df[standardized_df.groupby('user').user.transform('count')>4].copy()
master_df.head()

,user,song,count,analysis_songs_track_id,metadata_songs_artist_id,metadata_songs_song_id,analysis_songs_key,analysis_songs_key_confidence,analysis_songs_mode,analysis_songs_mode_confidence,...,tempo_binned,analysis_songs_duration,metadata_songs_artist_familiarity,metadata_songs_artist_hotttnesss,metadata_songs_song_hotttnesss,metadata_artist_terms,metadata_songs_artist_name,metadata_songs_release,metadata_songs_title,musicbrainz_artist_mbtags
3,724534729c9f5dc72a009269c2c225883e4775d2,SOWEZSI12A81C21CE6,1,TRAUCNU128F42671EB,AR2UQQ51187B9AC816,SOWEZSI12A81C21CE6,5,0.351,0,0.318,...,8,-0.280898,0.732042,1.243373,5.09644,b'flamenco'|b'soundtrack'|b'folk'|b'spanish'|b...,Gipsy Kings,Greatest Hits,Tu Quieres Volver,b'classic pop and rock'|b'folk'
4,ee7aa84c164038c963cfd02a7e52a5598aa470c3,SOWEZSI12A81C21CE6,2,TRAUCNU128F42671EB,AR2UQQ51187B9AC816,SOWEZSI12A81C21CE6,5,0.351,0,0.318,...,8,-0.280898,0.732042,1.243373,5.09644,b'flamenco'|b'soundtrack'|b'folk'|b'spanish'|b...,Gipsy Kings,Greatest Hits,Tu Quieres Volver,b'classic pop and rock'|b'folk'
5,1aa4fd215aadb160965110ed8a829745cde319eb,SOWEZSI12A81C21CE6,1,TRAUCNU128F42671EB,AR2UQQ51187B9AC816,SOWEZSI12A81C21CE6,5,0.351,0,0.318,...,8,-0.280898,0.732042,1.243373,5.09644,b'flamenco'|b'soundtrack'|b'folk'|b'spanish'|b...,Gipsy Kings,Greatest Hits,Tu Quieres Volver,b'classic pop and rock'|b'folk'
8,cca2694a19b2474bc731574ebb8b10382214f865,SOWEZSI12A81C21CE6,2,TRAUCNU128F42671EB,AR2UQQ51187B9AC816,SOWEZSI12A81C21CE6,5,0.351,0,0.318,...,8,-0.280898,0.732042,1.243373,5.09644,b'flamenco'|b'soundtrack'|b'folk'|b'spanish'|b...,Gipsy Kings,Greatest Hits,Tu Quieres Volver,b'classic pop and rock'|b'folk'
17,091d959cd1d21f270d9f6dc58b93ee78caa4ab96,SOWEZSI12A81C21CE6,10,TRAUCNU128F42671EB,AR2UQQ51187B9AC816,SOWEZSI12A81C21CE6,5,0.351,0,0.318,...,8,-0.280898,0.732042,1.243373,5.09644,b'flamenco'|b'soundtrack'|b'folk'|b'spanish'|b...,Gipsy Kings,Greatest Hits,Tu Quieres Volver,b'classic pop and rock'|b'folk'


In [8]:
model_df = master_df[['user', 'song', 'count']]
model_df.loc[:,'rating']=1
model_df.head()

,user,song,count,rating
3,724534729c9f5dc72a009269c2c225883e4775d2,SOWEZSI12A81C21CE6,1,1
4,ee7aa84c164038c963cfd02a7e52a5598aa470c3,SOWEZSI12A81C21CE6,2,1
5,1aa4fd215aadb160965110ed8a829745cde319eb,SOWEZSI12A81C21CE6,1,1
8,cca2694a19b2474bc731574ebb8b10382214f865,SOWEZSI12A81C21CE6,2,1
17,091d959cd1d21f270d9f6dc58b93ee78caa4ab96,SOWEZSI12A81C21CE6,10,1


In [6]:
## Get list of all unique songs
all_songs = model_df.song.unique()

## Placeholders for training data
users, items, labels = [], [], []

## The set of items that each user has interacted with
user_item_set = set(zip(model_df['user'], model_df['song']))

                    
## 4:1 ratio of negative to positive samples
num_negatives = 4

for (u, i) in tqdm(user_item_set):
    users.append(u)
    items.append(i)
    labels.append(1)
    for _ in range(num_negatives):
        negative_item = np.random.choice(all_songs)
        while(u, negative_item) in user_item_set:
            negative_item = np.random.choice(all_songs)
            users.append(u)
            items.append(negative_item)
            labels.apppend(0)
    
    
                

In [ ]:
standardized_df.user_id = standardized_df.user_id.astype('category').cat.codes.values
standardized_df.item_id = standardized_df.item_id.astype('category').cat.codes.values

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(standardized_df, test_size=0.2)